In [6]:
# Using finBert to analyse bitcoin news scape data
# Convert articles to word embeddings
# Get LSTM data same period of time
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [7]:
# %pip install requests beautifulsoup4
# %pip install selenium


In [8]:
import requests
from bs4 import BeautifulSoup

In [4]:
# https://www.coindesk.com/search?s=bitcoin&sd=1672596621000&ed=1672596621000&df=Custom

# The date style in the URL is Unix timestamp in milliseconds
# 1672596621000 corresponds to January 1, 2023, 12:03:41 AM GMT

# To convert a date like '2023-01-01' to this format:
from datetime import datetime
import time

date_str = '2023-01-02'
date_obj = datetime.strptime(date_str, '%Y-%m-%d')
unix_timestamp_ms = int(time.mktime(date_obj.timetuple()) * 1000)

print(f"Unix timestamp in milliseconds for {date_str}: {unix_timestamp_ms}")

Unix timestamp in milliseconds for 2023-01-02: 1672614000000


In [12]:

from datetime import datetime, timedelta
import json

def scrape_coindesk(start_date, end_date):
    base_url = "https://www.coindesk.com/search?s=bitcoin"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    current_date = start_date
    results = []
    
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)

    while current_date <= end_date:
        curr_date_str = current_date.strftime('%Y-%m-%d')
        date_unix = int(time.mktime(current_date.timetuple()) * 1000)
        url = f"{base_url}&sd={date_unix}&ed={date_unix}&df=Custom"
        driver.get(url)
    
        # Wait for the search results to load
        try:
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CLASS_NAME, "searchstyles__SearchCardWrapper-sc-ci5zlg-21"))
            )
        except:
            print(f"Timeout waiting for search results on {current_date}")
            continue
        
        html_content = driver.page_source
        soup = BeautifulSoup(html_content, 'html.parser')
        
        body_content = soup.find('body')
        if body_content:
            print("Body available")
        else:
            print("Body tag not found")
        
        articles = soup.find_all('div', {"class": 'searchstyles__SearchCardWrapper-sc-ci5zlg-21 jjGJEL'}, limit=6)
        print(f"All articles for {current_date}: {len(articles)}")
        for article in articles:
            title_element = article.find('h6', class_='typography__StyledTypography-sc-owin6q-0 dtjHgI')
            title = title_element.text.strip() if title_element else 0
            
            description_element = article.find('p', class_='typography__StyledTypography-sc-owin6q-0')
            description = description_element.text.strip() if description_element else 0
            
            results.append({
                "date": curr_date_str,
                "title": title,
                "description": description
            })
        
        current_date += timedelta(days=1)
    
    return results

# Example usage
start_date = datetime(2019, 1, 1)


end_date = datetime(2019, 1, 2)

scraped_data = scrape_coindesk(start_date, end_date)

# Print the results
# print(json.dumps(scraped_data, indent=2))
print(f"Seccessfully scraped {len(scraped_data)} data from {start_date} to {end_date}")

Body available
All articles for 2019-01-01 00:00:00: 1
Body available
All articles for 2019-01-02 00:00:00: 5
Seccessfully scraped 6 data from 2019-01-01 00:00:00 to 2019-01-02 00:00:00


In [13]:
scraped_data

[{'date': '2019-01-01', 'title': 0, 'description': 0},
 {'date': '2019-01-02', 'title': 0, 'description': 0},
 {'date': '2019-01-02',
  'title': 'Down More than 70% in 2018, Bitcoin Closes Its Worst Year on Record',
  'description': "Bitcoin's price just finished its worst performing year ever, closing 2018 at a more than 70 percent lower price than its yearly open."},
 {'date': '2019-01-02',
  'title': 'Bitcoin Price Suffers Worst Monthly Losing Streak in 7 Years',
  'description': 'Bitcoin fell for the fifth month in a row in December, confirming its longest monthly losing streak since November 2011.'},
 {'date': '2019-01-02',
  'title': 'E-Commerce Giant DMM Quits Cryptocurrency Mining Business',
  'description': 'Japanese e-commerce giant DMM.com is in the process of winding down its cryptocurrency mining business due to the crypto market slump.'},
 {'date': '2019-01-02',
  'title': 'The Secret for a $1 Trillion Crypto Market? Keep Building',
  'description': "Sure, the crypto mark

In [7]:
df = pd.DataFrame(scraped_data)
df.to_csv('./dataset/news-coindesk.csv')